<a href="https://colab.research.google.com/github/SBilaal/Cards/blob/master/Grain_Eating_Birds_Classification%20-%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data = '/content/gdrive/MyDrive/Birds_Test_Dataset'
train_path = f'{data}/train'
valid_path = f'{data}/valid'
test_path = f'{data}/test'
classes = ['AFRICAN FIREFINCH', 'CROWNED PIGEON', 'GREEN JAY', 'MOURNING DOVE', 'NICOBAR PIGEON', 'PURPLE FINCH', 'RED BROWED FINCH', 'ROCK DOVE', 'STRAWBERRY FINCH', 'YELLOW HEADED BLACKBIRD']
print(train_path)

os.path.isdir(train_path)
os.path.isdir(test_path)
os.path.isdir(valid_path)

/content/gdrive/MyDrive/Birds_Test_Dataset/train


True

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input).flow_from_directory(directory=train_path, target_size=(299,299), classes=classes, batch_size=8)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input).flow_from_directory(directory=valid_path, target_size=(299,299), classes=classes, batch_size=8)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input).flow_from_directory(directory=test_path, target_size=(299,299), classes=classes, batch_size=8, shuffle=False)

Found 950 images belonging to 10 classes.
Found 230 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


In [ ]:
imgs, labels = next(train_batches)

In [ ]:
def plotImages(images_arr):
  fig, axes = plt.subplots(1, 10, figsize=(20,20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
plotImages(imgs)
print(labels)

In [ ]:
xception = tf.keras.applications.xception.Xception()

91897856/91884032 [==============================] - 1s 0us/step


In [ ]:
xception.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
x = xception.layers[-2].output
output = Dense(units=10, activation='softmax')(x)

In [ ]:
model = Model(inputs=xception.input, outputs=output)

In [ ]:
for layer in model.layers[:-23]:
  layer.trainable = False

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=30, verbose=2)

Epoch 1/30
119/119 - 531s - loss: 0.7305 - accuracy: 0.8600 - val_loss: 0.0655 - val_accuracy: 0.9826
Epoch 2/30
119/119 - 522s - loss: 0.1257 - accuracy: 0.9779 - val_loss: 0.0253 - val_accuracy: 0.9957
Epoch 3/30
119/119 - 520s - loss: 0.0506 - accuracy: 0.9947 - val_loss: 0.0131 - val_accuracy: 1.0000
Epoch 4/30
119/119 - 523s - loss: 0.0309 - accuracy: 0.9968 - val_loss: 0.0166 - val_accuracy: 0.9957
Epoch 5/30
119/119 - 520s - loss: 0.0236 - accuracy: 0.9989 - val_loss: 0.0116 - val_accuracy: 1.0000
Epoch 6/30
119/119 - 516s - loss: 0.0205 - accuracy: 0.9989 - val_loss: 0.0164 - val_accuracy: 0.9957
Epoch 7/30
119/119 - 513s - loss: 0.0214 - accuracy: 0.9979 - val_loss: 0.0073 - val_accuracy: 1.0000
Epoch 8/30
119/119 - 513s - loss: 0.0086 - accuracy: 1.0000 - val_loss: 0.0070 - val_accuracy: 1.0000
Epoch 9/30
119/119 - 514s - loss: 0.0063 - accuracy: 0.9989 - val_loss: 0.0095 - val_accuracy: 1.0000
Epoch 10/30
119/119 - 519s - loss: 0.0066 - accuracy: 0.9989 - val_loss: 0.0192 - 

In [ ]:
test_imgs, test_labels = next(test_batches)
plotImages(test_imgs)
print(test_labels)

In [ ]:
test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches, verbose=2)

In [ ]:
np.round(predictions)

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=-72)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
print(test_batches.class_indices)
print(classes)

In [ ]:
cm_plot_labels = classes
plot_confusion_matrix(cm=cm, classes=classes, normalize=True, title='Confusion Matrix')